### 스케일 공간 이론
해리스 특징점은 이동과 회전에 불변이지만, 스케일에는 불변하지 않습니다.

스케일 공간(scale space) 이론은 스케일 불변의 가능성을 열었습니다.

스케일 공간에서 특징점을 검출하는 알고리즘의 흐름은 아래와 같습니다.
1. 입력 영상 $f$로부터 다중 스케일 영상 $\tilde{f}$를 구성한다.
2. $\tilde{f}$에 적절한 미분 연산을 적용하여 다중 스케일 미분 영상 $\tilde{f}'$를 구한다.
3. $\tilde{f}'$에서 극점을 찾아 특징점으로 취한다.

다중 스케일 영상 $\tilde{f}$은 가까이부터 멀리까지 본 장면을 표현해야 합니다.

이를 구현하는 첫 번째 방법은 거리가 멀어지면 세부 내용이 점점 흐려지는 현상을 모방합니다.
* 표준편차 $\sigma$를 점점 키우면서 가우시안 필터로 입력 영상을 스무딩하여 흐려지게 하면서 이를 구현합니다.

두번째 방법은 거리가 멀어지면 물체의 크기가 작아지는 현상을 모방합니다.
* 이는 영상의 크기를 반씩 줄인 영상들을 쌓은 피라미드 영상으로 이를 구현합니다.

피라미드 방법은 영상을 연속적인 스케일로 줄일 수 없는 한계가 있지만, 가우시안 스무딩은 $\sigma$를 연속된 값으로 조절할 수 있습니다.

1. 가우시안 방법으로 $\sigma$를 촘촘하게 적용하여 만든 영상들을 $t$ 축에 배치하여 다중 스케일 영상 $\tilde{f}$를 구성합니다.
2. $\tilde{f}$에 정규 라플라시안을 사용하여 미분 연산을 적용합니다.
3. 극점 검출은 주로 비최대 억제를 사용합니다. 스케일 공간은 3차원 이므로 3차원에서의 비최대 억제를 적용합니다.

# SIFT 알고리즘
스케일 공간에서 특징점을 검출하는 알고리즘은 여러가지가 있지만, 가장 성공적인 버전은 SIFT 알고리즘이다. 아래는 알고리즘의 주요 과정입니다.

### 1단계: 다중 스케일 영상 구축
SIFT는 가우시안 스무딩과 피라미드 방법을 결합해 다중 스케일 영상을 구성합니다.

1. 원본 영상에 $\sigma$를 다르게 적용한 6장의 사진을 만듭니다. 이런 영상들의 집합을 옥타브(octave)라 부릅니다.
2. 원본 영상을 반으로 줄이고 1번 과정을 반복합니다. 이렇게 만들어진 영상들은 '옥타브 1'이라고 할 수 있습니다.
3. 또다시 영상을 반으로 줄이고 반복합니다. 이렇게 옥타브 0, 1, 2, 3, ...를 만들면 다중 스케일 영상이 완성됩니다.

옥타브 0의 첫번째 영상은 원래 영상을 $\sigma_{1}=1.6$으로 스무딩한 영상에서 출발합니다.

두번째 영상은 $\sigma_{2}=k\sigma_{1}$로 스무딩하는데, 실험을 통해 $k=2^{1/3}$으로 정했습니다.

실제 구현에서는 원본 영상 자체가 이미 0.5만큼 스무딩되어 있다고 가정하고 $\sigma_{1}=1.6$ 대신 $\sigma_{1}=\sqrt{1.6^{2}-0.5^{2}}$을 사용합니다.

이렇게 $i+1$번째 영상은 $i$번째 영상에 $\sqrt{\sigma_{i+1}^{2}-\sigma_{i}^{2}}$로 컨볼루션하여 계산합니다.

결국 6번의 컨볼루션을 수행하여 옥타브 0을 구축할 수 있습니다. 비슷한 과정으로 옥타브 1, 2, 3, ...를 만듭니다.

### 2단계: 다중 스케일 영상에 미분 적용
SIFT는 미분 계산으로 정규 라플라시안을 사용하지만, 라플라시안은 큰 필터로 컨볼루션하므로 오랜 시간이 소요됩니다.

그래서 SIFT는 정규 라플라시안과 매우 유사하다고 증명된 DOG(Difference of Gaussian)를 사용하여 계산 시간을 획기적으로 줄입니다.

옥타브에 여섯 장의 영상이 있으면 최종적으로 다섯 장의 DOG 영상이 생깁니다.

### 3단계: 극점 검출
3차원 구조에서 화소의 극점 여부를 조사합니다.

$i$번째 DOG 영상에서 타겟 화소의 극점 여부를 조사하기 위해서는 $i$번째 영상에서 8개 이웃 화소, $i-1$번째의 9개 이웃 화소, $i+1$번째의 9개 이웃 화소와 비교합니다.

결론적으로 타겟 화소가 주변 26개 이웃 화소보다 크면 극점으로 인정하고 특징점으로 취합니다. SIFT는 특징점을 키포인트(keypoint)라고 부릅니다.

<img src="SIFT_DOG.jpg" alt="SIFT_DOG" width="1000"/>

검출된 특징점은 옥타브를 나타내는 $o$, DOG 번호를 나타내는 $i$, 옥타브에서 화소 위치를 나타내는 $y$와 $x$를 모아 $(y,x,o,i)$로 표현합니다.

$y$와 $x$는 위치 정보를 나타내고, $o$와 $i$는 스케일 정보를 나타낸다고 말할 수 있습니다.

이 $(y,x,o,i)$값들은 테일러 확장으로 미세 조정하여 최종 확정하고, 마지막으로 에지에서 검출된 특징점을 걸러내는 후처리 과정을 적용합니다.

### SIFT 기술자
SIFT 특징점의 위치와 스케일 정보만으로는 물체를 매칭하지 못하므로, 특징점 주위를 살펴 풍부한 정보를 가진 기술자(descriptor)를 추출해야 합니다.

기술자는 특징 벡터에 해당하는데, 특징점과 마찬가지로 기술자 역시 6개 조건을 만족해야 합니다.

특히 이동, 회전, 스케일, 조명 변화에 불변성을 만족해야 합니다.

키포인트 $(y,x,o,i)$의 $o$와 $i$ 정보를 이용해 가장 가까운 가우시안 영상을 결정하고 그 영상에서 기술자를 추출합니다. -> 스케일 불변성 달성

회전 불변성을 달성하려면 기준 방향을 정하고, 기준 방향을 중심으로 특징을 추출하면 됩니다.

* 기준 방향을 정하기 위해 키포인트 주위 영역에서 추출한 그레이디언트 분포를 이용합니다.
* 에지 연산자를 이용해 그레이디언트 강도와 방향을 계산합니다.
* 그레이디언트 방향을 10도 간격으로 양자화하여 36개 칸의 히스토그램을 구합니다.
* 히스토그램의 최대값을 지배적인 방향(dominant orientation)으로 정합니다.
* 최대값의 0.8배 이상인 값들도 지배적인 방향으로 취합니다. 따라서 하나의 키포인트에서 방향이 다른 여러 특징점이 발생할 수 있습니다.
* 이제 키포인트는 지배적인 방향 $\theta$를 추가하고 $o$와 $i$는 $\sigma$로 변환하여, $(y,x,\sigma,\theta)$로 표현됩니다.

키포인트에서 기술자를 추출하기 위해 $\theta$를 기준으로 윈도우를 씌우고, 보간하고 샘플링하여 16x16 크기의 작은 영역을 얻습니다.

그리고 이 영역을 4x4 크기의 블록 16개로 나눕니다.

각 블록은 자신에 속한 16개 화소의 그레이디언트 방향을 8단계(8가지 방향)로 양자화, 히스토그램을 구합니다. 이때, 가우시안을 가중치로 사용합니다.

블록이 16개 이고 블록마다 8차원 히스토그램이 만들어지므로 최종적으로 16x8=128차원의 기술자 $X$를 얻습니다.

SIFT는 조명 불변성을 달성하기 위해 기술자 $X$를 벡터의 크기 $\left\|X\right\|$로 나누어 단위 벡터로 바꿉니다.
* 또한 단위 벡터에 0.2보다 큰 요소가 있으면 0.2로 바꾼 다음 다시 단위 벡터로 바꿉니다.

> 이렇게 얻은 $X$가 최종 기술자가 됩니다. 이제 하나의 특징점(키포인트)는 $(y,x,\sigma,\theta,X)$로 표현됩니다.

> 입력 영상이 들어오면 SIFT는 특징점 집합 $(y_{i},x_{i},\sigma_{i},\theta_{i},X_{i}), i=1,2,...,n$을 출력합니다.